In [1]:
import numpy
import pandas
import statsmodels.api as sm
import seaborn
import statsmodels.formula.api as smf 
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pandas.set_option('display.float_format', lambda x:'%.2f'%x)

In [3]:
data = pandas.read_csv('/content/drive/My Drive/data/nesarc.csv', low_memory=False)

In [4]:
data['IDNUM'] =pandas.to_numeric(data['IDNUM'], errors='coerce')
data['TAB12MDX'] = pandas.to_numeric(data['TAB12MDX'], errors='coerce')
data['MAJORDEPLIFE'] = pandas.to_numeric(data['MAJORDEPLIFE'], errors='coerce')
data['NDSymptoms'] = pandas.to_numeric(data['NDSymptoms'], errors='coerce')
data['SOCPDLIFE'] = pandas.to_numeric(data['SOCPDLIFE'], errors='coerce')
data['S3AQ3C1'] = pandas.to_numeric(data['S3AQ3C1'], errors='coerce')
data['AGE'] =pandas.to_numeric(data['AGE'], errors='coerce')
data['SEX'] = pandas.to_numeric(data['SEX'], errors='coerce')
data['S3AQ3B1'] = pandas.to_numeric(data['S3AQ3B1'], errors='coerce')

sub1=data[(data['AGE']<=25) & (data['CHECK321']==1) & (data['S3AQ3B1']==1)]

In [ ]:
sub1['numbercigsmoked_c'] = (sub1['S3AQ3C1'] - sub1['S3AQ3C1'].mean())

sub1['age_c']=(sub1['AGE'] - sub1['AGE'].mean())

In [ ]:
# Mã hóa lại
sub1['ETHRACE2A'] = pandas.to_numeric(sub1['ETHRACE2A'], errors='coerce')
recode2 = {1: 1, 2: 2, 3: 3, 4: 3, 5: 0}
sub1['ETHRACE2A'] = sub1['ETHRACE2A'].replace(9, numpy.nan)
sub1['ETHRACE'] = sub1['ETHRACE2A'].map(recode2)

In [7]:
# Mô hình hồi quy
reg6 = smf.ols('NDSymptoms ~ DYSLIFE + MAJORDEPLIFE + numbercigsmoked_c + age_c + SEX + C(ETHRACE)', data=sub1).fit()
print (reg6.summary())

                            OLS Regression Results                            
Dep. Variable:             NDSymptoms   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     23.46
Date:                Wed, 04 Nov 2020   Prob (F-statistic):           7.76e-34
Time:                        09:09:07   Log-Likelihood:                -2613.4
No. Observations:                1320   AIC:                             5245.
Df Residuals:                    1311   BIC:                             5291.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.3243      0.18

In [ ]:
######################## HỒI QUY LOGISTIC ########################################

In [ ]:
sub1['S3AQ3B1']=sub1['S3AQ3B1'].replace(9, numpy.nan)
sub1['S3AQ3C1']=sub1['S3AQ3C1'].replace(99, numpy.nan)

recode1 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub1['USFREQMO']= sub1['S3AQ3B1'].map(recode1)
sub1['USFREQMO'] = pandas.to_numeric(sub1['USFREQMO'], errors='coerce')

sub1['NUMCIGMO_EST']=sub1['USFREQMO'] * sub1['S3AQ3C1']
sub1['NUMCIGMO_EST']= pandas.to_numeric(sub1['NUMCIGMO_EST'], errors='coerce')

In [9]:
sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()

In [10]:
log_reg = smf.logit(formula = 'MAJORDEPLIFE ~ NUMCIGMO_EST', data = sub2).fit()

Optimization terminated successfully.
         Current function value: 0.581005
         Iterations 5


In [11]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:           MAJORDEPLIFE   No. Observations:                 1315
Model:                          Logit   Df Residuals:                     1313
Method:                           MLE   Df Model:                            1
Date:                Wed, 04 Nov 2020   Pseudo R-squ.:                0.001198
Time:                        09:43:11   Log-Likelihood:                -764.02
converged:                       True   LL-Null:                       -764.94
Covariance Type:            nonrobust   LLR p-value:                    0.1758
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.1315      0.114     -9.920      0.000      -1.355      -0.908
NUMCIGMO_EST     0.0003      0.000      1.364      0.172      -0.000       0.001


In [13]:
### Đánh giá mô hình logistic
X = sub2[['NUMCIGMO_EST']]
y = sub2[['MAJORDEPLIFE']]

In [14]:
# Chia dữ liệu thành 2 tập: train và test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [15]:
log_reg2 = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.625513
         Iterations 5


In [18]:
yhat = log_reg2.predict(X_test) 
prediction = list(map(round, yhat))

In [19]:
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [20]:
cm = confusion_matrix(y_test, prediction)
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[295   0]
 [100   0]]


In [21]:
print('Test accuracy = ', accuracy_score(y_test, prediction))

Test accuracy =  0.7468354430379747


In [ ]:
############# TEST AGAIN ###########################

In [22]:
train, test = train_test_split(sub2, test_size=0.3, random_state=42, shuffle=True)

In [23]:
log_reg3 = smf.logit(formula = 'MAJORDEPLIFE ~ NUMCIGMO_EST', data = train).fit()

Optimization terminated successfully.
         Current function value: 0.587253
         Iterations 5


In [24]:
yhat = log_reg3.predict(test[['NUMCIGMO_EST']]) 
prediction = list(map(round, yhat))

In [25]:
cm = confusion_matrix(test[['MAJORDEPLIFE']], prediction)
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[295   0]
 [100   0]]


In [26]:
print('Test accuracy = ', accuracy_score(test[['MAJORDEPLIFE']], prediction))

Test accuracy =  0.7468354430379747
